# Homework №3

### <center>Student: *Valery Stoeva*</center>

# Description

1. Those how are making projects on artilces: half of you need to choose co-authorship graph, others - citation (but consider citation graph as undirected for simplicity)
2. Use weighted graph so that it would give you ability to calculate embeddings in more appropriate way

### Guidelines:

1. Initiallize your classification set as follows:
    * Determine training and testing intervals on your time domain (for example, take a period $2000$-$2014$ as training period and $2015$-$2018$ as testing period)
    * Pick pairs of nodes that **have appeared during training interval** but **had no links** during it
    * These pairs form **positive** or **negative** examples depending on whether they have formed coauthorships **during the testing interval**
    * You have arrived to binary classification problem.
2. Construct feature space:
    * Use at least 2 features based on neighborhood 
    * Use at least 2 fetures based on shortest path
    * Use embedding representation of nodes' pairs (for example, node2vec)
    * Use idea of time series features (with time lag)
    * Use idea of change-point detection
3. Choose at least $3$ classification algorithms and compare them in terms of Accuracy, Precision, Recall, F-Score (for positive class) and Mean Squared Error. Use k-fold cross-validation and average your results

Функция получения данных

In [1]:
import mysql.connector

mydb = mysql.connector.connect(
  host="articlesgrap.cxqhtfp4sprs.eu-central-1.rds.amazonaws.com",
  user="wizard",
  passwd="12345678",
  database="TheData"
)

def getAutGraph(by, fy):
    '''get co-authors between years by and fy'''
    mycursor = mydb.cursor()
    sql = "SELECT a.auid, b.auid, count(b.auid) as weight FROM TheData.isAuthor as a join TheData.isAuthor as b on a.artid=b.artid join TheData.Articles as c on a.artid=c.id where c.year between %s and %s and a.auid <> b.auid group by a.auid, b.auid order by a.auid desc;"
    val = (by, fy)
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()
    mycursor.close()
    return myresult

Возьмем период 2010-2013 за training period и 2014-2015 за testing period

In [2]:
import networkx as nx

In [3]:
train_select = getAutGraph(2010,2013)
train_G = nx.Graph()
train_G.add_weighted_edges_from(train_select)

In [4]:
test_select = getAutGraph(2014,2015)
test_G = nx.Graph()
test_G.add_weighted_edges_from(test_select)

Находим общие вершины и создаем копии обоих графов, содержащие только общие вершины.

In [5]:
common_nodes = train_G.nodes & test_G.nodes

In [6]:
nodes_to_delete_from_train = [node for node in train_G.nodes if node not in common_nodes]
train_G_filtered = train_G.copy()
train_G_filtered.remove_nodes_from(nodes_to_delete_from_train)

nodes_to_delete_from_test = [node for node in test_G.nodes if node not in common_nodes]
test_G_filtered = test_G.copy()
test_G_filtered.remove_nodes_from(nodes_to_delete_from_test)

In [7]:
del train_select, test_select

Ребра, для которых нам нужно будет делать предсказания:

In [8]:
edge_list = list(nx.non_edges(train_G_filtered))

Сначала посчитаем ответы и, для интереса, распределение ответов

Классы сильно несбалансированные, ну ничего.

## Фичи на основе соседей

Добавим по 2 фичи для вершины на основе ребер из вершины - количество и взвешенную сумму, и по таких же 2 фичи для вершины на основе ребер из соседей вершины. 5 фича для пары вершин - количество общих соседей

In [9]:
neighbour_sum, neighbour_weighted_sum, neighbour_edges_sum, neighbour_edges_weighted_sum, common_neighbours = {}, {}, {}, {}, {}
for node in train_G.nodes():
    neighbour_sum[node] = len(train_G[node])
    neighbour_weighted_sum[node] = sum(edge_dict['weight'] for edge_dict in train_G[node].values())
    
for node in train_G_filtered.nodes():
    neighbour_edges_sum[node] = sum(neighbour_sum[neighbour] for neighbour in train_G[node])
    neighbour_edges_weighted_sum[node] = sum(neighbour_weighted_sum[neighbour] for neighbour in train_G[node])

for edge in edge_list:
    common_neighbours[edge] = len(list(nx.common_neighbors(train_G, *edge)))

## Фичи на основе расстояния
Взвешенное и невзвешенное. Для взвешенного в каестве весов используем обратное число общих статей, чтобы расстояние было осмысленным. (при использовании не обратных общих статей при поиске кратчайших путей получается какая-то странная величина)

Тут небольшое допущение - будем искать данные пути не на исходном графе, а на отфильтрованном (только из общих вершин обоих периодов), т.к. сложность флойда-Уоршелла кубическая; но может это и верно - зачем учитывать авторов, которые не появляются в обоих периодах.

In [10]:
for edge, d in train_G_filtered.edges.items():
    d['opp_weight'] = 1 / d['weight']

distances = {}
weighted_distances = {}


for cc in nx.connected_component_subgraphs(train_G_filtered):
    distances.update(nx.floyd_warshall(cc))
    weighted_distances.update(nx.floyd_warshall(cc, weight='opp_weight'))

## Фичи на основе изменений харастеристик во времени
Для каждого года обучающей выборки и для каждой вершины найдем изменение ее степени

In [11]:
time_series = {node: [] for node in train_G_filtered.nodes()}

for year in range(2010, 2013 + 1):
    temp_select = getAutGraph(year, year)
    temp_G = nx.Graph()
    temp_G.add_weighted_edges_from(temp_select)
    for node in train_G_filtered.nodes():
        if node in temp_G.nodes:
            time_series[node].append(len(temp_G[node]))
        else:
            time_series[node].append(0)

## Фичи на основе точек изменения тренда
Будем называть год точкой изменения, если за этот год было написано n статей, за предыдущий год k статей, и max(n,k)/(min(n,k)+1) > 2

In [12]:
trends_changes = {node: [] for node in train_G_filtered.nodes()}

for node, flags in trends_changes.items():
    prev = 0
    for delta in time_series[node]:
        flags.append(int(max(prev, delta) / (1 + min(prev, delta)) > 2))
        prev = delta

## Node2Vec

Неочевидно, какой из двух весов учитывать в Node2Vec'е, поэтому будем подавать невзвешанные ребра

In [13]:
with open('in.txt', 'w') as f:
    for edge in train_G.edges():
        f.write('{} {}\n'.format(*edge))

In [14]:
!python2 n2v/src/main.py --input in.txt --output out.txt --dimensions 32

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [15]:
nodes_to_vecs = {node: [] for node in train_G_filtered.nodes()}

with open('out.txt') as f:
    f.readline()
    for line in f.readlines():
        split_line = line.split()
        nodes_to_vecs[int(split_line[0])] = list(map(float, split_line[1:]))

## Собираем фичи в numpy-матрицу

In [16]:
import math

X = []
y = [edge in test_G_filtered.edges for edge in edge_list]

for edge in edge_list:
    features = []
    node_1, node_2 = edge
    
    # фичи соседства
    features.extend(
        (
            neighbour_sum[node_1],
            neighbour_sum[node_2],
            neighbour_weighted_sum[node_1],
            neighbour_weighted_sum[node_2],
            neighbour_edges_sum[node_1],
            neighbour_edges_sum[node_2],
            neighbour_edges_weighted_sum[node_1],
            neighbour_edges_weighted_sum[node_2],
            common_neighbours[edge],
        )
    )

    # фичи расстояния
    # да, корректно это будет работать только для деревьев, и, возможно, нейросетей
    features.extend(
        (
            distances[node_1].get(node_2, -1),
            weighted_distances[node_1].get(node_2, -1),
        )
    )

    # time-series
    features.extend(time_series[node_1])
    features.extend(time_series[node_2])

    # changing poins
    features.extend(trends_changes[node_1])
    features.extend(trends_changes[node_2])

    # node2vec
    features.extend(nodes_to_vecs[node_1])
    features.extend(nodes_to_vecs[node_2])

    X.append(features)

In [17]:
import numpy as np

X_matrix = np.array(X)
y_matrix = np.array(y)

X_matrix.shape

(131088, 91)

## Logistic Regression

Тут и во всех классификаторах(кроме байесовского) мы будем использовать сбалансированные веса для классов, т.к. классы несбалансированы - доля положительного всего лишь:

In [27]:
sum(y) / len(y)

0.0016096057610155011

In [18]:
from sklearn.model_selection import KFold

In [19]:
kf = KFold(n_splits=5)

In [20]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(class_weight='balanced')

lr_predicts = np.zeros(y_matrix.shape)

for train_index, test_index in kf.split(X_matrix):
    train_X, train_y = X_matrix[train_index], y_matrix[train_index]
    test_X, test_y = X_matrix[test_index], y_matrix[test_index]
    lr.fit(train_X, train_y)
    lr_predicts[test_index] = lr.predict(test_X)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

## Случайный лес

In [21]:
from sklearn.ensemble import RandomForestClassifier

rf_predicts = np.zeros(y_matrix.shape)

rf = RandomForestClassifier(n_estimators=100, class_weight='balanced')

for train_index, test_index in kf.split(X_matrix):
    train_X, train_y = X_matrix[train_index], y_matrix[train_index]
    test_X, test_y = X_matrix[test_index], y_matrix[test_index]
    rf.fit(train_X, train_y)
    rf_predicts[test_index] = rf.predict(test_X)

## LightGBM (градиентный бустинг)

In [23]:
from lightgbm import LGBMClassifier

lightgmb_predicts = np.zeros(y_matrix.shape)

gb = LGBMClassifier(objective='binary', class_weight='balanced')

for train_index, test_index in kf.split(X_matrix):
    train_X, train_y = X_matrix[train_index], y_matrix[train_index]
    test_X, test_y = X_matrix[test_index], y_matrix[test_index]
    gb.fit(train_X, train_y)
    lightgmb_predicts[test_index] = gb.predict(test_X)

## Naive Bayes Classifier

In [24]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb_predicts = np.zeros(y_matrix.shape)

for train_index, test_index in kf.split(X_matrix):
    train_X, train_y = X_matrix[train_index], y_matrix[train_index]
    test_X, test_y = X_matrix[test_index], y_matrix[test_index]
    nb.fit(train_X, train_y)
    nb_predicts[test_index] = nb.predict(test_X)

## Нейросеть

In [25]:
from tensorflow import keras
import tensorflow as tf
from sklearn.utils import class_weight

nn = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

nn_predicts = np.zeros(y_matrix.shape)

for train_index, test_index in kf.split(X_matrix):
    train_X, train_y = X_matrix[train_index], y_matrix[train_index]
    test_X, test_y = X_matrix[test_index], y_matrix[test_index]
    
    cws = {i: w for i, w in enumerate(class_weight.compute_class_weight('balanced', np.unique(train_y), train_y))}
    
    nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    nn.fit(train_X, train_y, batch_size=len(train_X), epochs=100, class_weight=cws, verbose=2)
    nn_predicts[test_index] = nn.predict_classes(test_X).flatten()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 1s - loss: 7.4170 - acc: 0.9983
Epoch 2/100
 - 0s - loss: 6.8875 - acc: 0.9975
Epoch 3/100
 - 3s - loss: 5.8625 - acc: 0.9569
Epoch 4/100
 - 1s - loss: 5.6634 - acc: 0.7434
Epoch 5/100
 - 1s - loss: 5.6994 - acc: 0.6808
Epoch 6/100
 - 1s - loss: 5.6826 - acc: 0.6368
Epoch 7/100
 - 1s - loss: 5.6184 - acc: 0.6068
Epoch 8/100
 - 1s - loss: 5.4913 - acc: 0.5790
Epoch 9/100
 - 1s - loss: 5.3256 - acc: 0.5541
Epoch 10/100
 - 0s - loss: 5.0936 - acc: 0.5289
Epoch 11/100
 - 0s - loss: 4.7437 - acc: 0.5010
Epoch 12/100
 - 1s - loss: 4.4902 - acc: 0.4650
Epoch 13/100
 - 1s - loss: 4.2243 - acc: 0.4272
Epoch 14/100
 - 0s - loss: 3.8848 - acc: 0.3920
Epoch 15/100
 - 0s - loss: 3.3541 - acc: 0.3524
Epoch 16/100
 - 0s - loss: 2.5730 - acc: 0.3345
Epoch 17/100
 - 0s - loss: 2.0299 - acc: 0.4755
Epoch 18/100
 - 0s - loss: 2.2570 - acc: 0.5064
Epoch 19/100
 - 0s - loss

Epoch 61/100
 - 0s - loss: 0.8770 - acc: 0.8265
Epoch 62/100
 - 0s - loss: 0.8861 - acc: 0.8308
Epoch 63/100
 - 0s - loss: 0.8895 - acc: 0.7774
Epoch 64/100
 - 0s - loss: 0.8766 - acc: 0.7917
Epoch 65/100
 - 0s - loss: 0.8812 - acc: 0.8435
Epoch 66/100
 - 0s - loss: 0.8620 - acc: 0.8300
Epoch 67/100
 - 0s - loss: 0.8652 - acc: 0.8181
Epoch 68/100
 - 0s - loss: 0.8562 - acc: 0.8497
Epoch 69/100
 - 1s - loss: 0.8625 - acc: 0.8644
Epoch 70/100
 - 0s - loss: 0.8662 - acc: 0.8071
Epoch 71/100
 - 0s - loss: 0.8575 - acc: 0.8127
Epoch 72/100
 - 0s - loss: 0.8634 - acc: 0.8540
Epoch 73/100
 - 1s - loss: 0.8473 - acc: 0.8180
Epoch 74/100
 - 1s - loss: 0.8504 - acc: 0.8019
Epoch 75/100
 - 1s - loss: 0.8388 - acc: 0.8379
Epoch 76/100
 - 1s - loss: 0.8562 - acc: 0.8590
Epoch 77/100
 - 1s - loss: 0.8496 - acc: 0.7986
Epoch 78/100
 - 0s - loss: 0.8491 - acc: 0.7950
Epoch 79/100
 - 0s - loss: 0.8449 - acc: 0.8488
Epoch 80/100
 - 1s - loss: 0.8297 - acc: 0.8263
Epoch 81/100
 - 0s - loss: 0.8321 - acc:

 - 5s - loss: 0.8321 - acc: 0.8652
Epoch 33/100
 - 1s - loss: 0.8340 - acc: 0.8834
Epoch 34/100
 - 1s - loss: 0.8400 - acc: 0.8763
Epoch 35/100
 - 1s - loss: 0.8395 - acc: 0.8522
Epoch 36/100
 - 0s - loss: 0.8325 - acc: 0.8391
Epoch 37/100
 - 0s - loss: 0.8204 - acc: 0.8527
Epoch 38/100
 - 1s - loss: 0.8199 - acc: 0.8759
Epoch 39/100
 - 1s - loss: 0.8267 - acc: 0.8851
Epoch 40/100
 - 1s - loss: 0.8200 - acc: 0.8777
Epoch 41/100
 - 0s - loss: 0.8142 - acc: 0.8616
Epoch 42/100
 - 1s - loss: 0.8157 - acc: 0.8561
Epoch 43/100
 - 1s - loss: 0.8160 - acc: 0.8679
Epoch 44/100
 - 1s - loss: 0.8136 - acc: 0.8875
Epoch 45/100
 - 1s - loss: 0.8087 - acc: 0.8952
Epoch 46/100
 - 0s - loss: 0.8053 - acc: 0.8843
Epoch 47/100
 - 1s - loss: 0.8068 - acc: 0.8681
Epoch 48/100
 - 0s - loss: 0.8069 - acc: 0.8622
Epoch 49/100
 - 1s - loss: 0.8029 - acc: 0.8715
Epoch 50/100
 - 0s - loss: 0.8011 - acc: 0.8861
Epoch 51/100
 - 0s - loss: 0.8016 - acc: 0.8914
Epoch 52/100
 - 0s - loss: 0.7994 - acc: 0.8832
Epoch

## Сравнение классификаторов

In [26]:
from sklearn import metrics

def evaluate(true_y, pred_y):
    print(
        'Accuracy: {}\nPrecision: {}\nRecall: {}\nF-Score: {}\nMSE: {}'.format(
        metrics.accuracy_score(true_y, pred_y),
        metrics.precision_score(true_y, pred_y),
        metrics.recall_score(true_y, pred_y),
        metrics.f1_score(true_y, pred_y),
        metrics.mean_squared_error(true_y, pred_y),
        )
    )

In [28]:
evaluate(y_matrix, lr_predicts)

Accuracy: 0.7897290369827902
Precision: 0.00398940993000399
Recall: 0.5213270142180095
F-Score: 0.007918226317304926
MSE: 0.21027096301720982


Логистическая регрессия - самый низкий precision, видимо слишком много у нас FP (судя по accuracy, если пренебречь долей положительного класса, то 20%), зато неплохой recall! Все-таки мы пытаемся балансировать классы, и регрессия пытается половину положительных детектировать правильно, хоть и вместе с этим 20% нулевых записываются в положительные.

Precision всего в 2 раза большое, чем доля положительного класса в выборке.

In [29]:
evaluate(y_matrix, rf_predicts)

Accuracy: 0.9984132796289515
Precision: 0.5789473684210527
Recall: 0.052132701421800945
F-Score: 0.09565217391304347
MSE: 0.001586720371048456


Никакой recall и хороший precision - лес почти каждый объект старается отнести к 0 классу, даже 95% положительных объектов туда отнес, и только 5% положительных объектов он отнес правильно, и столько же к 1 классу он отнес элементов 0 класса.

In [30]:
evaluate(y_matrix, lightgmb_predicts)

Accuracy: 0.9972232393506651
Precision: 0.23529411764705882
Recall: 0.3222748815165877
F-Score: 0.27199999999999996
MSE: 0.002776760649334798


Самая высокая F-мера! Несмотря на то, что по другим категориям другие классификаторы его обошли, F-мера в данной ситуации более репрезентативна, поэтому пока LGBM побеждает 

In [31]:
evaluate(y_matrix, nb_predicts)

Accuracy: 0.9918833150250214
Precision: 0.03691639522258415
Recall: 0.16113744075829384
F-Score: 0.06007067137809187
MSE: 0.00811668497497864


Как логистическая регрессия, только хуже

In [32]:
evaluate(y_matrix, nn_predicts)

Accuracy: 0.8870758574392774
Precision: 0.008554492792671427
Recall: 0.6018957345971564
F-Score: 0.0168692302583516
MSE: 0.11292414256072257


Самый большой recall, точность в 5 раз выше концентрации 1 класса, топ-2 F-мера. Нейросети - вторые после LGBM.